In [1]:
import sys
import json
import csv
import pandas as pd
import time


#Make sure you have all the above libraries and openpyxl xlsxwriter xlrd 
#e.g. via pip install

# encoding=utf8
#This stuff below is because some abstracts have weird things in them 
#Also affected by python 2 vs 3
from __future__ import unicode_literals
import sys
reload(sys)
sys.setdefaultencoding('utf8')


In [2]:
#Export the JSON file from Indico:
# - Go into management mode
# - Click on "Call for Abstracts" on the left column under organization
# - On "List of Abstracts" click "Manage"
# - Check all the abstracts
# - Under "Export" click "JSON"


#Read JSON file
f = open('abstracts.json',) 
  
# returns JSON object as a dictionary 
abstract_data = json.load(f) 
f.close() 

#Make a dictionary of titles so we have linked JSON id # and title from schedule
title_dict = {}

for i_d in range(len(abstract_data['abstracts'])): 
    title_dict.update({abstract_data['abstracts'][i_d]['title']:i_d})
    
    #Examples of various things in the JSON file
    print(abstract_data['abstracts'][i_d]['title']) #Talk title
    print(abstract_data['abstracts'][i_d]['persons'][0]['title']) #Prof., Mr. Ms...
    print(abstract_data['abstracts'][i_d]['persons'][0]['first_name'])
    print(abstract_data['abstracts'][i_d]['persons'][0]['last_name'])
    print(abstract_data['abstracts'][i_d]['persons'][0]['affiliation'])
    print(abstract_data['abstracts'][i_d]['content'])
    print(abstract_data['abstracts'][i_d]['persons'][0]['email'])
    print(abstract_data['abstracts'][i_d]['persons'][0]['id']) #abstract id from indico

    print("\n")



In [3]:

#Make this schedule file yourself using identical talk titles for as the JSON file
#Copy format of the 2021 excel for simplest implementation 
#The 2021 Excel Schedule is on the Overleaf for the 2021 book in the Scripts/ folder. 

#Read schedule as an excel file using pandas, make sure it is in the same directory
schedule_data = pd.read_excel('20210104-WNPPC-schedule.xlsx')

#print(schedule_data.keys()) #Take a look at all the keys in the excel sheet

# creating some blank series to use to fill in the dataframe 
email_series = pd.Series([])
affil_series = pd.Series([])
abstract_series = pd.Series([])
abstract_id_series = pd.Series([])

# running a for loop and asigning Email, affiliation, Abstract to the dataframe 
for i in range(len(schedule_data)): 
    if schedule_data["Title"][i] in title_dict: 
        email_series[i]= abstract_data['abstracts'][title_dict[schedule_data["Title"][i]]]['persons'][0]['email']
        affil_series[i]= abstract_data['abstracts'][title_dict[schedule_data["Title"][i]]]['persons'][0]['affiliation']
        abstract_series[i]= abstract_data['abstracts'][title_dict[schedule_data["Title"][i]]]['content']
        abstract_id_series[i] = abstract_data['abstracts'][title_dict[schedule_data["Title"][i]]]['persons'][0]['id']

#Add in Email, Affiliation and Abstract from JSON i_d which is linked to talk titles
schedule_data.insert(5, "Email", email_series)
schedule_data.insert(6, "Affiliation", affil_series)
schedule_data.insert(7, "Abstract", abstract_series)
schedule_data.insert(8, "abstract_id", abstract_id_series)


#Test if it looks right
#print((schedule_data.loc[1]))


In [4]:

#LaTeX output for the abstracts.tex chapter of the book of abstracts
abstracts = open("abstracts.tex", "w")

abstracts.write("""
%Generated from abstracts-chapter.ipynb
\\newpage
\\chapter{Abstracts}
\\renewcommand{\\thepage}{\\arabic{page}}% Arabic page numbers
\\setcounter{page}{1}
""")

#Begin for loop for abstracts

#Fix conditionals for handling NaNs and coffee breaks...

for i in range(len(schedule_data["Title"])-1):
    #print(schedule_data.loc[i].Time.strftime("%H:%M"))
    
    #Skip over inconvenient nan values...
    if( (type(schedule_data["Author"][i]) != float)
      and type(schedule_data["Time"][i]) != float):
        #print(str(schedule_data["Author"][i]))
        
        abstracts.write("""

        \\begin{conf-abstract}[%s \\\%s]
        {%s}
        {%s}
        {%s}

        %ss

        \end{conf-abstract}
                        """%(schedule_data.loc[i].Date,
                             schedule_data.loc[i].Time.strftime("%H:%M"), 
                             #schedule_data.loc[i+1].Time.strftime("%H:%M"), 
                             schedule_data.loc[i].Title,
                             schedule_data.loc[i].Author,
                             schedule_data.loc[i].Affiliation,
                             schedule_data.loc[i].Abstract,
                             ))
        if (type(schedule_data.loc[i].Email) != float): #avoiding empty emails
            abstracts.write("""
            {Abstract ID: %d \\\ 
            
            contact: %s}
                            """%(schedule_data.loc[i].abstract_id,schedule_data.loc[i].Email))
            
        #new page for new abstract
        abstracts.write("""
        \\newpage
        """) 
          
        

#End for loop for abstracts.tex file

abstracts.close()

#Make sure to check the abstracts.tex file manually and do some clean up. 
